In [1]:
from mindquantum import *

In [2]:
H

H

In [3]:
H.on(0)

H(0)

In [4]:
H.matrix()

array([[ 0.70710678,  0.70710678],
       [ 0.70710678, -0.70710678]])

In [5]:
X.matrix()

array([[0, 1],
       [1, 0]])

In [7]:
X.on(1, [0,2,3])

X(1 <-: 0 2 3)

In [10]:
import numpy as np
RX(np.pi).on(0, 1)

RX(π|0 <-: 1)

In [11]:
RX('theta').on(1)

RX(theta|1)

In [12]:
circuit = Circuit()

In [13]:
circuit = Circuit([H.on(0), X.on(1, 0)])
circuit

q0: ──H────●──
           │  
q1: ───────X──

In [14]:
circuit += qft(range(3))
circuit

q0: ──H────●────H────PS(π/2)────PS(π/4)─────────────────────────@──
           │            │          │                            │  
q1: ───────X────────────●──────────┼───────H────PS(π/2)─────────┼──
                                   │               │            │  
q2: ───────────────────────────────●───────────────●───────H────@──

In [15]:
circuit += ZZ('alpha').on([0,1])
circuit

q0: ──H────●────H────PS(π/2)────PS(π/4)─────────────────────────@────ZZ(alpha)──
           │            │          │                            │        │      
q1: ───────X────────────●──────────┼───────H────PS(π/2)─────────┼────ZZ(alpha)──
                                   │               │            │               
q2: ───────────────────────────────●───────────────●───────H────@───────────────

In [16]:
sim = Simulator('projectq', 3)

In [17]:
sim

projectq simulator with 3 qubits.
Current quantum state:
1¦000⟩

In [18]:
sim.apply_circuit(qft(range(3)))
sim

projectq simulator with 3 qubits.
Current quantum state:
√2/4¦000⟩
√2/4¦001⟩
√2/4¦010⟩
√2/4¦011⟩
√2/4¦100⟩
√2/4¦101⟩
√2/4¦110⟩
√2/4¦111⟩

In [19]:
sim.reset()

In [20]:
sim

projectq simulator with 3 qubits.
Current quantum state:
1¦000⟩

In [21]:
encoder = Circuit()
encoder += H.on(0)
encoder += RX('alpha0').on(0)
encoder += RY('alpha1').on(0)
encoder += RZ('alpha2').on(0)
encoder

q0: ──H────RX(alpha0)────RY(alpha1)────RZ(alpha2)──

In [24]:
print(encoder.get_qs('projectq', {'alpha0': 0.1, 'alpha1':0.2, 'alpha2':0.3}, ket=True))

(0.6203638230723639-0.1257539726032947j)¦0⟩
(0.7702994677056821+0.07728774449984671j)¦1⟩


In [25]:
ansatz = Circuit()
ansatz += RX('theata0').on(0)
ansatz += RY('theta1').on(0)
ansatz

q0: ──RX(theata0)────RY(theta1)──

In [28]:
circuit = encoder + ansatz
circuit 

q0: ──H────RX(alpha0)────RY(alpha1)────RZ(alpha2)────RX(theata0)────RY(theta1)──

In [29]:
sim = Simulator('projectq', circuit.n_qubits)
sim

projectq simulator with 1 qubit.
Current quantum state:
1¦0⟩

In [33]:
ham = Hamiltonian(-QubitOperator('Z0'))
ham

-1.0 [Z0] 

In [37]:
grad_ops = sim.get_expectation_with_grad(ham, circuit,encoder_params_name=encoder.params_name, ansatz_params_name=ansatz.params_name)
grad_ops

In [38]:
grad_ops(np.array([[0.1, 0.2, 0.3]]), np.array([0.4,0.5]))

(array([[0.51048884+0.j]]),
 array([[[ 5.55111512e-17+0.j,  7.21265887e-01+0.j, -4.58830979e-01+0.j]]]),
 array([[[-0.30200415+0.j,  0.78801928+0.j]]]))

In [41]:
import mindspore as ms
ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target="CPU")
net = MQLayer(grad_ops)
net

MQLayer<
  (evolution): MQOps<1 qubit projectq VQA Operator>
  >

In [42]:
net(ms.Tensor(np.array([[0.1,0.2,0.3]])))

Tensor(shape=[1, 1], dtype=Float32, value=
[[ 1.95401520e-01]])

In [46]:
opti = ms.nn.Adam(net.trainable_params(), learning_rate=0.5)
train_net = ms.nn.TrainOneStepCell(net, opti)
train_net

TrainOneStepCell<
  (network): MQLayer<
    (evolution): MQOps<1 qubit projectq VQA Operator>
    >
  (optimizer): Adam<>
  >

In [47]:
for i in range(100):
    res = train_net(ms.Tensor(np.array([[0.1,0.2,0.3]])))
    print(res)

[WARNING] DEBUG(2676,7f53e7697740,python):2021-10-21-07:43:37.579.633 [mindspore/ccsrc/debug/debugger/debugger.cc:88] Debugger] Not enabling debugger. Debugger does not support CPU.
[[0.19540152]]
[[-0.4202797]]
[[-0.86169165]]
[[-0.9592955]]
[[-0.83309096]]
[[-0.7295181]]
[[-0.7280006]]
[[-0.7987455]]
[[-0.8862574]]
[[-0.9385713]]
[[-0.93191373]]
[[-0.88537765]]
[[-0.84084976]]
[[-0.8276558]]
[[-0.849409]]
[[-0.89079744]]
[[-0.9295071]]
[[-0.9485769]]
[[-0.9454192]]
[[-0.931335]]
[[-0.9217197]]
[[-0.9259928]]
[[-0.9441032]]
[[-0.9687466]]
[[-0.98992205]]
[[-0.9997197]]
[[-0.9959427]]
[[-0.98301625]]
[[-0.96936786]]
[[-0.9626718]]
[[-0.9659615]]
[[-0.97675836]]
[[-0.9891862]]
[[-0.99750197]]
[[-0.999035]]
[[-0.9950435]]
[[-0.98930883]]
[[-0.985645]]
[[-0.9859139]]
[[-0.98947835]]
[[-0.99404925]]
[[-0.99722975]]
[[-0.99785256]]
[[-0.99641305]]
[[-0.99447435]]
[[-0.99356276]]
[[-0.9943063]]
[[-0.99625844]]
[[-0.99834275]]
[[-0.999544]]
[[-0.9994489]]
[[-0.99838984]]
[[-0.99717027]]
[[-0.

In [48]:
net.trainable_params()[0].asnumpy()

array([ 2.174837, -1.207377], dtype=float32)